<a href="https://colab.research.google.com/github/ik339/NTU-R-language/blob/main/Bioinformatical_analysis_of_genome_data_R_language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Sequences were collected to begin the analysis: 
#•	reference.fasta (Wuhan reference sequence (NC_045512.2) for SARS-CoV-2) 
#•	minkn.fasta (mink-derived SARS-CoV-2 genome sequence)
#•	euroseq.fasta (sequence data), euroseq.csv (metadata) . These are data from other countries. colinear files)

#First import the correct libraries into R.
> library(tidyverse)
> library(lubridate)
> library(stringdist)
> library(Biostrings)
#change working directory in R to where the sequences are located on the computer.
> setwd("/home/biouser/Downloads/assessment")


In [ ]:
#Import csv file into a named tibble.
> euroseqMetaData <- read_csv("euroseq.csv")
# parse tibble using tidyverse functions:
> ymd(euroseqMetaData$Collection_Date)


In [ ]:
 #The following code was used to derive a vector corresponding to samples with incompletely formatted collection dates 
 #and/or non-human hosts and then remove this data: 
> IncorrectDates <- is.na(ymd(euroseqMetaData$Collection_Date))  #Boolean vector identifying incomplete dates.
> notHumanHost <- euroseqMetaData %>% select(Host) %>% as_vector() %>% str_detect("Felis") #Boolean vector identifying Felis catus 
> NotHumanIncompleteDates <-  IncorrectDates | notHumanHost #combined Boolean vectors.


In [ ]:
> euroseqMDFiltered <- euroseqMetaData %>% filter(!NotHumanIncompleteDates) #Make new tibble. Incorrect dates and non-human hosts removed.
> euroseqMDFiltered %>% type_convert() #Parse again
> ymd(euroseqMDFiltered$Collection_Date)  #check things are parsed correctly. 
> rm(“euroseqMetaData”) #remove old tibble 


In [ ]:
> euroseqMDFiltered %>% arrange(Collection_Date)
> euroseqMDFiltered %>% filter(Collection_Date == max(Collection_Date))   #Identification of the recent collected date


In [ ]:
> euroseqMDFNew <- euroseqMDFiltered %>% mutate(Days = ymd(20200514) - ymd(Collection_Date)) %>% filter(Days >= 0)
#Subtracting each row's collection date from this recent collection point. 
#Permanently add column “Days”, meaning days before recent collection date. 

In [ ]:
#import fasta into a named object.
> euroseqFastaStringSet <- readDNAStringSet("euroseq.fasta", format = "fasta")

In [ ]:
#as the data is colinear remove the same data using the aleady created combined Boolean vector. 
> euroseqFSSFilt <- euroseqFastaStringSet[!NotHumanIncompleteDates]
> rm("euroseqFastaStringSet") #remove old unfiltered object.


In [ ]:
> euroseqfasta  <- names(euroseqFSSFilt) %>% str_split(" \\|", n=2, simplify = TRUE) #parse sequence names from object.
> colnames(euroseqfasta) <- c("Country", "Accession")	#add column names.
> euroseqNewfasta<- euroseqfasta %>% as_tibble()  #create new tibble.


In [ ]:
> euroseqFBA <- euroseqNewfasta %>% mutate(BaseAcc = str_extract(Accession, "[A-Za-z0-9_]+"))	 
#permenantly adds base accession column to the tibble.

In [ ]:
> euroDataJoined <- euroseqFBA%>%full_join(euroseqMDFNew, by = c("BaseAcc"="Accession")) 
# full outer join executed

In [ ]:
#modify sequence names in the,object to contain country, accession, host species and days prior to last sample
> names(euroseqFSSFilt) <- str_c(names(euroseqFSSFilt), "|", euroDataJoined$Host)
> names(euroseqFSSFilt) <- str_c(names(euroseqFSSFilt), "|", euroDataJoined$Days) #add new collumns. 
> names(euroseqFSSFilt)%>%as_tibble()

In [ ]:
#writing object into named fasta file
> writeXStringSet(euroseqFSSFilt, "euroseqFSSFilt.fasta", format = "fasta")


In [ ]:
#open bash terminal 
conda activate msa 
cd Downloads/assessment
cat euroFSSFilt.fasta mink3.fasta > combined.fasta #Combine fasta and mink sequences.


In [ ]:
#using MAFFT carry out a reference-based alignment
mafft --auto --addfragments combined.fasta reference.fasta > aligned.fasta #open mafft

In [ ]:
#Using the microseq library aligned sequence data was trimmed to remove all gaps at the 5’ and 3’ ends of the alignment.
library(microseq) #load library in R
setwd("/home/biouser/Downloads/assessment")
Alignment <- readFasta("aligned.fasta")  #read it in R.
dim(Alignment) #check dimensions
str_length(Alignment$Sequence) #check length
TrimmedAlignment <- msaTrim(Alignment, gap.end=0, gap.mid=1) #trim str_length(TrimmedAlignment$Sequence)#check length.

#The reference sequence was removed and data written to file: 
WithoutReference <- TrimmedAlignment[2:dim(TrimmedAlignment)
writeFasta(WithoutReference, "trimmed.fasta", width = 80)

#Aligned and trimmed data were then used as input for phylogenetic analysis.
